論文<br>
<br><br>
HuggingFace<br>
https://huggingface.co/DionTimmer/controlnet_qrcode-control_v1p_sd15<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/QRCode_AIArt_Generator_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup environment

## git clone

In [ ]:
# @markdown Initialize image
init_img_url = 'https://user0514.cdnw.net/shared/img/thumb/shikunDJI_0204_TP_V4.jpg?' #@param {type:"string"}
# @markdown Initialize image use or not
use_init_img = False #@param {type:"boolean"}
# @markdown URL for QR code
target_url = 'https://www.12-technology.com/' #@param {type:"string"}
# @markdown random seed
seed = 12 #@param {type:"integer"}
# @markdown text prompt
text_prompt = 'Sky view of highly aesthetic, ancient greek thermal baths  in beautiful nature' #@param {type:"string"}
# @markdown image size
img_size = 768 #@param {type:"integer"}
# @markdown output path
output_path = './result.jpg' #@param {type:"string"}

## install libraries

In [ ]:
%cd /content

!pip -q install diffusers transformers accelerate torch xformers
!pip install qrcode

## import libraries

In [ ]:
%cd /content

import torch
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, DDIMScheduler, DPMSolverMultistepScheduler
from diffusers.utils import load_image

import qrcode

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device is', device)

# Load pretrained model from huggingface

In [ ]:
controlnet_id = 'DionTimmer/controlnet_qrcode-control_v1p_sd15'
stable_diffusion_id = 'runwayml/stable-diffusion-v1-5'

controlnet = ControlNetModel.from_pretrained(
    controlnet_id,
    torch_dtype=torch.float16)

pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    stable_diffusion_id,
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.enable_xformers_memory_efficient_attention()
# pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, use_karras=True, algorithm_type="sde-dpmsolver++")
pipe.enable_model_cpu_offload()

# Setup image

In [ ]:
init_path = './init_img.jpg'
qr_img_path = './qr_code_img.jpg'

## init image

In [ ]:
# Init image
!rm -rf {init_path}

!wget -c {init_img_url} \
      -O {init_path}

In [ ]:
# 正方形になるようにCrop
im = Image.open(init_path)
print(im.size)

x1, y1, x2, y2 = 0, 0, 532, 532
im_croped = im.crop((x1, y1, x2, y2))
im_croped.save(init_path)

In [ ]:
plt.imshow(plt.imread(init_path))
plt.axis('off')
plt.show()

## init qr code

In [ ]:
img = qrcode.make(target_url)
img.convert("RGB").save(qr_img_path)

In [ ]:
plt.imshow(plt.imread(qr_img_path))
plt.axis('off')
plt.show()

# Define func

In [ ]:
def resize_for_condition_image(input_image: Image, resolution: int):
  input_image = input_image.convert("RGB")
  W, H = input_image.size
  k = float(resolution) / min(H, W)
  H *= k
  W *= k
  H = int(round(H / 64.0)) * 64
  W = int(round(W / 64.0)) * 64
  img = input_image.resize((W, H), resample=Image.LANCZOS)
  return img

# Generate QR Code

In [ ]:
# qr code image
source_image = load_image(qr_img_path)
# initial image, anything
init_image = load_image(init_path)

In [ ]:
condition_image = resize_for_condition_image(source_image, img_size)
if use_init_img:
  image = resize_for_condition_image(init_image, img_size)
else:
  image = condition_image

generator = torch.manual_seed(seed)

image = pipe(
    prompt                            = text_prompt,
    negative_prompt                   = "ugly, disfigured, low quality, blurry, nsfw",
    image                             = image,
    control_image                     = condition_image,
    width                             = img_size,
    height                            = img_size,
    guidance_scale                    = 7.5,
    controlnet_conditioning_scale     = 1.5,
    generator                         = generator,
    strength                          = 0.9,
    num_inference_steps               = 50
    )

In [ ]:
image.images[0].save(output_path)

plt.imshow(plt.imread(output_path))
plt.axis('off')
plt.show()